# Example of Inference and Evaluation using the eICU.sqlite Database

 eICU Collaborative Research Database, a multi-center database comprised of deidentified health data for over 200,000 admissions to ICUs across the United States between 2014-2015. The database includes vital sign measurements, care plan documentation, severity of illness measures, diagnosis information, and treatment information.

Source of the dataset can be found [here](https://physionet.org/content/eicu-crd-demo/2.0.1/sqlite/)

In [26]:
import os
from openai import OpenAI
from loguru import logger
import sys
sys.path.append(os.path.abspath(os.path.join('..', 'model_evaluation')))


# Load test datsaet

## Load original test dasta

The test data used in our benchmark is stored in `vrdc_text2sql/model_evaluation/dataset/test/test_ehrsql_eicu_data.json` file. It has the following fields: 

- `index`
- `ddl`: DDL of the database
- `instructions`: Instruction for the LLM on how to respond
- `user`: actual user query
- `output`: ground truth SQL statement

In [27]:
import json

fields = ["index", "ddl", "instructions", "user_query", "output"]

test_data_fp = "../model_evaluation/dataset/test/test_ehrsql_eicu_data.json"
test_data = json.load(open(test_data_fp, "r"))


for data in test_data:
    for field in fields:
        if field not in data:
            print(f"Field {field} not found in data")
            break
        else:
            print(f"{field}:")
            print(f"{data[field]}\n")
    else:
        break
    break

index:
1

ddl:
DROP TABLE IF EXISTS patient;
CREATE TABLE patient
(
    uniquepid VARCHAR(10) NOT NULL,
    patienthealthsystemstayid INT NOT NULL,
    patientunitstayid INT NOT NULL PRIMARY KEY,
    gender VARCHAR(25) NOT NULL,
    age VARCHAR(10) NOT NULL,
    ethnicity VARCHAR(50),
    hospitalid INT NOT NULL,
    wardid INT NOT NULL,
    admissionheight NUMERIC(10,2),
    admissionweight NUMERIC(10,2),
    dischargeweight NUMERIC(10,2),
    hospitaladmittime TIMESTAMP(0) NOT NULL,
    hospitaladmitsource VARCHAR(30) NOT NULL,
    unitadmittime TIMESTAMP(0) NOT NULL,
    unitdischargetime TIMESTAMP(0),
    hospitaldischargetime TIMESTAMP(0),
    hospitaldischargestatus VARCHAR(10)
) ;

DROP TABLE IF EXISTS diagnosis;
CREATE TABLE diagnosis
(
    diagnosisid INT NOT NULL PRIMARY KEY,
    patientunitstayid INT NOT NULL,
    diagnosisname VARCHAR(200) NOT NULL,
    diagnosistime TIMESTAMP(0) NOT NULL,
    icd9code VARCHAR(100),
    FOREIGN KEY(patientunitstayid) REFERENCES patient(pati

In [28]:
# number of data points in the test set

print("Number of data points in the test set:", len(test_data))

Number of data points in the test set: 1792


## Create a mini test set with 10 samples

For demo purpose, we will use the first 10 examples 

In [29]:
# create a mini test set of 10 data points for demo purposes
size = 10
mini_testset = test_data[:size]

# create a new file for the mini test set
mini_testset_fp  = "../model_evaluation/dataset/test/test_ehrsql_eicu_data_mini.json"
with open(mini_testset_fp, "w") as f:
    json.dump(mini_testset, f, indent=4)


In [30]:
# load the mini test set
mini_testset_fp  = "../model_evaluation/dataset/test/test_ehrsql_eicu_data_mini.json"
mini_testset = json.load(open(mini_testset_fp, "r"))

# print the first record
print(json.dumps(mini_testset[0], indent=4))

{
    "index": 1,
    "ddl": "DROP TABLE IF EXISTS patient;\nCREATE TABLE patient\n(\n    uniquepid VARCHAR(10) NOT NULL,\n    patienthealthsystemstayid INT NOT NULL,\n    patientunitstayid INT NOT NULL PRIMARY KEY,\n    gender VARCHAR(25) NOT NULL,\n    age VARCHAR(10) NOT NULL,\n    ethnicity VARCHAR(50),\n    hospitalid INT NOT NULL,\n    wardid INT NOT NULL,\n    admissionheight NUMERIC(10,2),\n    admissionweight NUMERIC(10,2),\n    dischargeweight NUMERIC(10,2),\n    hospitaladmittime TIMESTAMP(0) NOT NULL,\n    hospitaladmitsource VARCHAR(30) NOT NULL,\n    unitadmittime TIMESTAMP(0) NOT NULL,\n    unitdischargetime TIMESTAMP(0),\n    hospitaldischargetime TIMESTAMP(0),\n    hospitaldischargestatus VARCHAR(10)\n) ;\n\nDROP TABLE IF EXISTS diagnosis;\nCREATE TABLE diagnosis\n(\n    diagnosisid INT NOT NULL PRIMARY KEY,\n    patientunitstayid INT NOT NULL,\n    diagnosisname VARCHAR(200) NOT NULL,\n    diagnosistime TIMESTAMP(0) NOT NULL,\n    icd9code VARCHAR(100),\n    FOREIGN K

# Generate SQL queries

In [31]:
import asyncio
from openai import AsyncOpenAI
import json
from loguru import logger

# Re-using vLLM server details from previous cells
IP = "localhost"
PORT = 8000
BASE_URL = f"http://{IP}:{PORT}/v1"
MODEL_PATH = "/home/ubuntu/workspace/mistral-nemo-minitron-8b-instruct-healthcare-text2sql_vV2.8"

# Initialize the AsyncOpenAI client
async_client = AsyncOpenAI(
    api_key="not-needed",
    base_url=BASE_URL,
)

In [32]:
from utils import postprocess_sql_query_from_markdown

async def get_prediction(record: dict) -> str:
    """
    Generates a SQL query for a given record asynchronously.
    """
    index = record['index']
    ddl = record['ddl']
    instruction = record['instructions']
    question = record['user_query']
    output = record['output'] # ground truth SQL statement

    prompt_chat_template = [
        {
            "role": "system",
            "content": f"Based on DDL statements, instructions, generate a SQL query to answer the user's question.\n\nDDL statements:\n{ddl}\nInstructions:\n{instruction}",
        },
        {
            "role": "user",
            "content": f"{question}"
        }
    ]

    try:
        response = await async_client.chat.completions.create(
            model=MODEL_PATH,
            messages=prompt_chat_template,
            temperature=0.0,
            max_tokens=512,
            stop=["<extra_id_1>"]
        )
        generated_sql = response.choices[0].message.content
        prediction = postprocess_sql_query_from_markdown(generated_sql)
        logger.info(f"Generated SQL for index {index}: {prediction}") 

        return {
            "index": index,
            "input": question, # user query
            "predict": prediction,  # model predicted SQL
            "real": output # ground truth SQL
        }
        
    except Exception as e:
        logger.error(f"An error occurred for index {record.get('index', 'N/A')}: {e}")
        return f"Error: {e}"

In [33]:
async def run_predictions_async(test_fp: str, output_fp: str):
    """
    Loads a test set and runs predictions asynchronously.
    """
    # Path to the mini test set
    with open(test_fp, "r") as f:
        test_data = json.load(f)

    tasks = [get_prediction(record) for record in test_data]
    
    logger.info(f"Sending {len(tasks)} requests to the vLLM server...")
    predictions = await asyncio.gather(*tasks)
    logger.success("All predictions done.")

    # add predictions to the test data by looking up the index
    results = []
    for pred in predictions:
        # create a dictionary with the index and prediction
        results.append({
            "index": pred['index'],
            "input": pred['input'],
            "predict": pred['predict'], 
            "real": pred['real']
        })

    # handle if the output file does not exist
    with open(output_fp, "w") as f:
        json.dump(results, f, indent=2)

    logger.success(f"Predictions saved to {output_fp}")

In [34]:
from utils import preprare_directory
from time import time

# create output directory. set exist_ok=False to delete the old results and re-create a fresh folder
output_dir = "../model_predictions/eICU_mini"
preprare_directory(output_dir, exist_ok=False)

# input test set directory
test_dir = "../model_evaluation/dataset/test"

start_time = time()
# run predictions
await run_predictions_async(
    test_fp = os.path.join(test_dir, "test_ehrsql_eicu_data_mini.json"),
    output_fp = os.path.join(output_dir, "predictions.json")
)
end_time = time()
logger.info(f"Time taken: {end_time - start_time:.2f} seconds")

2025-07-27 03:49:06.113 | INFO     | __main__:run_predictions_async:11 - Sending 10 requests to the vLLM server...
2025-07-27 03:49:06.877 | INFO     | __main__:get_prediction:34 - Generated SQL for index 2: select distinct medication.routeadmin from medication where medication.drugname = 'clonidine';
2025-07-27 03:49:07.132 | INFO     | __main__:get_prediction:34 - Generated SQL for index 3: select distinct medication.routeadmin from medication where medication.drugname = 'morphine 2 mg/1 ml 1 ml syr';
2025-07-27 03:49:07.260 | INFO     | __main__:get_prediction:34 - Generated SQL for index 4: select distinct medication.routeadmin from medication where medication.drugname = 'propofol 1000 mg/100 ml (pmx)';
2025-07-27 03:49:07.566 | INFO     | __main__:get_prediction:34 - Generated SQL for index 7: select distinct cost.cost from cost where cost.eventtype = 'Procedure' and cost.eventid in ( select treatment.treatmentid from treatment where treatment.treatmentname = 'internal fixation' )

In [35]:
# the prediction result is stored in a JSON file. The content looks like this

results = json.load(open(os.path.join(output_dir, "predictions.json"), "r"))

# pretty print the JSON results
for result in results:
    print(json.dumps(result, indent=4))
    break

{
    "index": 1,
    "input": "tell me the method of dextrose 5% in water (d5w) iv : 1000 ml bag intake?",
    "predict": "select distinct treatment.treatmentname from treatment where treatment.treatmentname = 'Dextrose 5% in Water (D5W) IV : 1000 ml bag';",
    "real": "select distinct medication.routeadmin from medication where medication.drugname = 'dextrose 5% in water (d5w) iv : 1000 ml bag'"
}


where: 
- `index`: same `index` as in the raw data
- `input`: user question
- `predict`: text2SQL model's predicted SQL statement
- `real`: ground turth SQL statement

# Evaluate SQL query

To evaluate the SQL query, we will execute BOTH the ground truth SQL statement and the predicted SQL statement against the SQL database, and compare their outcomes

### What correct means



Thes script considers a prediction to be correct (`accuracy=1`) if: 
1. execution results (sorted first 100 rows) are the same between ground truth SQL statement execution and predicted SQL statement execution, OR
2. both ground truth and predicted SQL statement execution failed, OR
3. ground truth SQL execution failed, and predcted SQL statement execution returns empty. 

```python
sample_scores['accuracy'] = 1 if real_result == pred_result or both_failed or real_failed_pred_empty else 0
```


### Metrics


The metrics are: 

```python
    out_eval = OrderedDict([
        ('precision_ans', round(100.0 * precision_ans, args.ndigits)),
        ('recall_ans', round(100.0 * recall_ans, args.ndigits)),
        ('f1_ans', round(100.0 * f1_ans, args.ndigits)),
        ('precision_exec', round(100.0 * precision_exec, args.ndigits)),
        ('recall_exec', round(100.0 * recall_exec, args.ndigits)),
        ('f1_exec', round(100.0 * f1_exec, args.ndigits)),
        ('acc', round(100.0 * acc, args.ndigits)),
    ])
```

They can be grouped into two main categories: **Answerability Metrics** (`_ans`) and **Execution Metrics** (`_exec`), plus overall accuracy.

#### 1. Answerability Metrics (`_ans`)

These metrics **ignore the correctness of the SQL query**. They only care about whether the model correctly identified if a question was *answerable* or *unanswerable*. **A question is considered answerable if its ground-truth query can be executed without error.**

- `precision_ans`: **"Of all the questions the model *claimed* were answerable, what percentage actually were?"**
    - This measures how much you can trust the model when it generates a query. A high score means it doesn't often generate queries for questions that are impossible to answer.
- `recall_ans`: **"Of all the questions that *truly were* answerable, what percentage did the model *attempt* to answer?"**
    - This measures how comprehensive the model is. A high score means the model doesn't frequently give up on questions it should have been able to answer 
- `f1_ans`: This is the **F1-score for answerability**. 
    - It's the harmonic mean of `precision_ans` and `recall_ans`, providing a single score that balances the two. It gives a good overall sense of how well the model distinguishes answerable from unanswerable questions.

#### 2. Execution Metrics (`_exec`)

These are the most important metrics. They evaluate the model's performance on the questions that are **supposed to be answerable**. They check if the *result* of the predicted SQL query matches the *result* of the ground-truth SQL query. 

- `precision_exec`: **"Of all the answerable questions the model *attempted* to answer, what percentage of its SQL queries were correct?"**
    - This is a direct measure of the model's correctness. When the model generates a query, how often is that query right?
- `recall_exec`: **"Of all the answerable questions in the dataset, what percentage did the model answer correctly?"**
    - This measures how much of the total task the model accomplished successfully. It penalizes the model for both failing to answer a question it should have and for getting the answer wrong.
- `f1_exec`: This is the **F1-score for execution**. 
    - It's the harmonic mean of `precision_exec` and `recall_exec`. This is often considered the primary metric for evaluating the overall quality of the text-to-SQL model.

#### 3. Overall Accuracy (`acc`)

`acc`: **"What is the overall percentage of questions that the model got right?"**
- This is the simplest metric. It considers a prediction correct if the result matches the ground-truth result, or if both correctly failed (i.e., for unanswerable questions). It provides a good, high-level summary of performance across all questions, both answerable and unanswerable.

### Run evaluation

For evaluation of the eICU SQLite database, we have a script:  `model_evaluation/ehrsql_eval.py`

Arguments: 

- `--pred_file`: Path to model predictions (JSON/JSONL)
- `--db_path`: Path to SQLite database file
- `--num_workers`: Parallel workers (default: -1 for all CPUs)
- `--timeout`: Query execution timeout (seconds)
- `--out_file`: Output directory for results and metrics
- `--ndigits`: Number of digits to round scores (default: 2)

In [41]:
import os
from utils import preprare_directory

# create output directory for evaluation results, relative to the path of model_evaluation directory
# note that the evaluate results need a clean new folder, because it will overwrite any existing files in the folder
pred_directory = "../model_predictions/eICU_mini"
eval_directory = os.path.join(pred_directory, "evaluation")
preprare_directory(eval_directory, exist_ok=False)

# the predicted file from previous step
pred_file = os.path.join(pred_directory, "predictions.json")

# path to the eICU database
db_path = "../model_evaluation/databases/eicu.sqlite"


In [42]:
# run evaluation
!python ../model_evaluation/ehrsql_eval.py \
    --pred_file {pred_file} \
    --db_path {db_path} \
    --num_workers -1 \
    --timeout 60 \
    --out_file {eval_directory} \
    --ndigits 2

## Interpret results

The `predictions_evaluation_results.jsonl` contains line by line execution results: 

In [49]:
fp = "../model_predictions/eICU_mini/evaluation/predictions_evaluation_results.jsonl"

# load jsonl file
with open(fp, "r") as f:
    for line in f:
        result = json.loads(line)
        print(json.dumps(result, indent=4))
        break

{
    "index": 1,
    "input": "tell me the method of dextrose 5% in water (d5w) iv",
    "predict": "select distinct treatment.treatmentname from treatment where treatment.treatmentname = 'Dextrose 5% in Water (D5W) IV : 1000 ml bag';",
    "real": "select distinct medication.routeadmin from medication where medication.drugname = 'dextrose 5% in water (d5w) iv : 1000 ml bag'",
    "pred": "select distinct treatment.treatmentname from treatment where treatment.treatmentname = 'dextrose 5% in water (d5w) iv : 1000 ml bag';",
    "real_result": "[\"('iv',)\"]",
    "pred_result": "[]",
    "sample_scores": {
        "precision_ans": 1,
        "precision_exec": 0,
        "recall_ans": 1,
        "recall_exec": 0,
        "accuracy": 0
    }
}


The `../model_predictions/eICU_mini/evaluation/predictions_metrics.json` contains the overal execution result

In [50]:
fp = "../model_predictions/eICU_mini/evaluation/predictions_metrics.json"

with open(fp, "r") as f:
    metrics = json.load(f)

print(json.dumps(metrics, indent=4))

{
    "precision_ans": 100.0,
    "recall_ans": 100.0,
    "f1_ans": 100.0,
    "precision_exec": 40.0,
    "recall_exec": 40.0,
    "f1_exec": 40.0,
    "acc": 40.0
}
